# Scrapper Políticos 

**Requisitos**: Se debe instalar el paquete de selenium con "pip install selenium" y correr el código de abajo. Si saliera un error, solo cerrar todas las ventanas de Jupyter Notebook y re lanzar el Jupyter. Con eso, se debería abrir una nueva ventana en el navegador en donde se ejecutará el programa. Para que el programa funcione, es necesario tener una conexión a internet estable. El programa se ejecutará desde un navegador Chrome.

## Imports de librerías 

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from webdriver_manager.firefox import GeckoDriverManager

from bs4 import BeautifulSoup
import re
import requests
from urllib.request import FancyURLopener
import pandas as pd
import csv
import math
import numpy as np
import unidecode
import re 
import time

from itertools import cycle
import traceback
import openpyxl

## Leer la base de datos para conseguir la información de búsqueda de los políticos

#### Celdas a cambiar

En caso de que la línea 1 bote error, simplemente reemplazar esa línea por esta línea
- df = pd.read_csv("l_distrital.csv", encoding = "latin-1",sep=',')

In [3]:
df = pd.read_csv("l_distrital.csv", encoding = "utf-8",sep=',')
df = df.replace(np.nan, '', regex=True)

## Proceso que obtiene los links de las hojas de vida de los políticos de la base de datos

Si es que hubieran problemas para acceder al servidor general, este código también puede ser implementado para las búsquedas en el mismo navegador de políticos. El código ha comenzado la recuperación completa de links el 23/09/2019 a las 16:18pm

#### Celdas a cambiar 

- Lo que debe ir entre paréntesis (de rojo) en el campo <b> key1 </b> es el nombre de la celda que contiene los nombres de los candidatos
- Lo que debe ir entre paréntesis (de rojo) en el campo <b> key2 </b> es el nombre de la celda que contiene el apellido paterno de los candidatos
- Lo que debe ir entre paréntesis (de rojo) en el campo <b> key3 </b> es el nombre de la celda que contiene el apellido materno de los candidatos
- Lo que debe ir entre paréntesis (de rojo) en el campo <b> key4 </b> es el nombre de la celda que contiene el año de elección de los candidatos

En la línea 110, cambiar el texto  entre paréntesis (de rojo) para crear un nuevo archivo. Por ejemplo, cambiar el texto original de "links_raw" a "links_raw1"

In [8]:
sess = requests.Session()
adapter = requests.adapters.HTTPAdapter(max_retries = 20)
sess.mount('https://', adapter)

In [37]:
real_years = []
year_real_cv = []
candidato = 2
for candidato in range(0, len(df)):
    # 1. Recopilación de Nombres, Apellidos y año de elección de la base de datos


    key1 = df["dist_forenames"][candidato]
    key2 = df["dist_first_lastname"][candidato]
    key3 = df["dist_second_lastname"][candidato]
    key4 = df["election_year"][candidato]

    # 2. Búsqueda de políticos

    driver = webdriver.Chrome()
    driver.get("https://infogob.jne.gob.pe/Politico")
    driver.maximize_window()


    nombre =  WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "TxNombre")))
    nombre.clear()
    nombre.send_keys(key1)
    WebDriverWait(driver, 15)

    apellido_p = driver.find_element_by_name("TxApePat")
    apellido_p.clear()
    apellido_p.send_keys(key2)
    WebDriverWait(driver, 15)

    apellido_m = driver.find_element_by_name("TxApeMat")
    apellido_m .clear()
    apellido_m .send_keys(key3)

    buscar = driver.find_element_by_id("btnBuscar")
    buscar.click()

    # 3. Resultados de la búsqueda

    WebDriverWait(driver, 30)

    politico0 = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "politico0")))
    politico =  WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.ID, "politico0")))
    politico.click()

    # 4. Lista desplegable de hojas de vida de un político en específico

    driver.find_element_by_xpath('//*[@id="section"]/div[2]/div[3]/div[3]/div[1]/div')
    hv = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[3]/div[1]/div')))
    hv.click()

    # 5. Comparación de años de la hoja de de vida con años de la base de datos proporcionada

    url = driver.current_url
    content = sess.get(url).content

    # 6. Recuperación de años del listbox

    soup = BeautifulSoup(content,'lxml')
    list_tags = soup.findAll('ul', attrs = {"class" : "lista-hojavida float-right"})

    # 7. Tratamiento de los tags encontrados para conseguir los años

    h_vida = []
    c_years = []

    for li in list_tags:
        h_vida.extend(li.findAll('span'))
        years = list(dict.fromkeys(h_vida))

    for e in range (0, len(years)):
        element = int(years[e].text[-4:])
        c_years.append(element)

    hh_vida = []
    for hey in range (0, len(h_vida)):
        hh_vida.append(h_vida[hey].text)



    # 8. Hacer click en la hoja de vida del año correcto


    year = key4
    print(year)
    right_year = dict(zip(hh_vida, c_years))
    keys = list(right_year.keys())
    values = list(right_year.values())
    try:
        r_year = values.index(year)
    except:
        r_year = None

    if r_year == None:
        mas_cercano = h_vida[-1]
        e_year = keys[-1]
        real_years.append(e_year)
        fecha_cv = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.LINK_TEXT, e_year))).click()
        year_real_cv.append(e_year)
    else:
        real_years.append(values[r_year])
        fecha_cv = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.LINK_TEXT, keys[r_year]))).click()
        year_real_cv.append(keys[r_year])


    # 9. Camino adicional para obtener los links
        if r_year == 2006 or e_year == 2006:
            frame = driver.find_elements_by_xpath('//*[@id="mdViewPdf"]')
            src = driver.find_element_by_id("mdViewPdf").get_attribute("src")
        else:
            frame = driver.find_elements_by_xpath('//*[@id="ifViewPdf"]')
            src = driver.find_element_by_id("ifViewPdf").get_attribute("src")


        driver.switch_to.frame(frame[0])

        driver.switch_to.default_content()

        # 10. Descarga de los links de las hojas de vida de los políticos

        with open("links_raw.csv",'a', newline='') as hojas_vida:
            wr = csv.writer(hojas_vida)
            wr.writerow(src)

        WebDriverWait(driver, 100)
        driver.quit()

2006
2014


KeyboardInterrupt: 

## Trasformación de links y recopilacion de años

 Si el nombre "links_raw" cambió, entonces deberá se reemplazar el primer paréntesis (de rojo) con el nombre de archivo nuevo. Por ejemplo, si el archivo de la celda anterior ya no se llama "links_raw", sino "links_raw_1", entonces el nombre en el primer paréntesis (de rojo) deberá tener por nombre "links_raw_1" 

In [5]:
with open("links_raw.csv") as etapa1, open("hojas_procesado.csv", "w") as etapa2:
    for enlace in etapa1:
        etapa2.write(enlace.replace(",", ""))

clean_links = pd.read_csv("hojas_procesado.csv", encoding = "latin-1", sep=';', header=None)
clean_links = clean_links.dropna()

cv_year = []

for link in range(0, len(clean_links)):
    test = clean_links[0][link]
    if (re.search('2006', test) != None) or (re.search('infogob', test) != None):
        cv_year.append(2006)
    elif re.search('2010', test) != None:
        cv_year.append(2010)
    elif re.search('==', test) != None:
        cv_year.append(2014)
    else:
        cv_year.append('clasificacion manual')

clean_links['cv_year'] = cv_year
clean_links.rename(columns={0:'links'}, inplace=True)
clean_links['links'] = clean_links['links'].str.replace("https://", "http://")

for i in range (0, len(clean_links)):
    if clean_links["cv_year"][i] == 2014:
        clean_links['links'][i] = clean_links['links'][i].replace("http://", "https://")

### Catalogar los links por año para agilizar el recojo de datos

In [6]:
y2010 = clean_links.loc[clean_links['cv_year'] == 2010]
y2006 = clean_links.loc[clean_links['cv_year'] == 2006]
y2014 = clean_links.loc[clean_links['cv_year'] == 2014]
other = clean_links.loc[clean_links['cv_year'] == 'clasificacion manual']

C:\Users\Giuliana\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [10]:
y2014

,links,cv_year


### Pasar los años catalogados a archivos csv independientes

In [ ]:
y2006.to_csv('2006_links.csv', sep=",", encoding='utf-8', index = False,  mode='a')
y2010.to_csv('2010_links.csv', sep=",", encoding='utf-8', index = False,  mode='a')
y2014.to_csv('2014_links.csv', sep=',', encoding='utf-8', index = False,  mode='a')
other.to_csv('other_links.csv', sep=',', encoding='utf-8', index = False,  mode='a')

## Aquisión de la información solicitada acerca de los políticos

### Variables

1. organizacion_politica : Organización Política por la que postulaba el candidato al momento de la elección
2. cargo: Cargo al que postulaba el político en el año de la elección
3. lugar_postulacion : Lugar compuesto por departamento, provincia y distrito en el cual el candidato postulaba.
4. DNI : El documento de identidad del candidato en formato texto
5. apellido_paterno : El apellido paterno del candidato
6. apellido_materno : El apellido materno del candidato
7. nombres : Nombres compeltos del candidato
8. fecha_nacimiento: día, mes y año en el que nació el candidato. Expresado en guiones de tipo DD/MM/YYYY en formato texto
9. sexo : Sexo del candidato
10. pais: Lugar de nacimiento del candidato
11. departamento_nac : Departamento de nacimiento del candidato
12. provincia_nac : Provincia de nacimiento del candidato
13. distrito_nac : Distrito de nacimiento del candidato
14. departamento_dom : Departamento en el que vive (domiciliado) el candidato
15. provincia_dom : Provincia en la que vive (domiciliado) el candidato
16. distrito_dom : Distrito en el que vive (domiciliado) el candidato
17. direccion : Dirección del candidato a la fecha
18. tiempo_dom : Tiempo de residencia en la provincia, distrito y departamento en el que vive actualmente el candidato
19. trabajo_publico: Respuesta a la pregunta ¿El candidato alguna vez ha traabajado en el sector público? Posibles respuestas: 'si', 'no'
20. trabajo_privado: Respuesta a la pregunta ¿El candidato alguna vez ha traabajado en el sector privado? Posibles respuestas: 'si', 'no'
21. publico_periodo : Número de años completos en los que el cantidato trabajó en el sector público
22. privado_periodo : Número de años completos en los que el cantidato trabajó en el sector privado
23. primaria : Respuesta a la pregunta ¿Concluyó la primaria? Posibles respuestas 'concluido', 'no concluido'
24. secundaria : Respuesta a la pregunta ¿Concluyó la secundaria? Posibles respuestas 'concluido', 'no concluido'
25. superior : Respuesta a la pregunta ¿Concluyó su educación superior? Posibles respuestas 'concluido', 'no concluido'
26. periodo_superior: Número de años completos en los que el candidato estuvo realizando estudios superiores
27. organizacion_politica_partidarios: Organización política en la que tuvo un cargo partidario, si es que tuvo uno
28. cargo_partidarios: Cargo partidario que ocupó, si es que tuvo uno
29. periodo_partidario: Número de años completos en los que tuvo un cargo partidario, si es que tuvo uno
30. organizacion_politica_popular: Organización política en la que tuvo un cargo de elección popular, si es que tuvo uno
31. cargo_popular: Cargo de elección popular que ocupó, si es que tuvo uno
32. periodo_popular: Número de años completos en los que tuvo un cargo de elección popular, si es que tuvo uno

In [ ]:
columns = ['organizacion_politica', 
           'cargo', 
           'lugar_postulacion',
           'DNI',
           'apellido_paterno',
           'apellido_materno',
           'nombres',
           'fecha_nacimiento',
           'sexo',
           'pais',
           'departamento_nac',
            'provincia_nac',
           'distrito_nac',
           'domicilio',
           'departamento_dom',
            'provincia_dom',
           'distrito_dom',
           'tiempo_dom',
          'trabajo_publico', 
           'trabajo_privado', 
           'publico_periodo', 
           'privado_periodo',
           'primaria', 
           'secundaria', 
           'superior', 
           'periodo_superior', 
          'organizacion_politica_partidarios',
           'cargo_partidarios',
           'periodo_partidarios',
           'organizacion_politica_popular', 
           'cargo_popular',
           'periodo_popular']

### Para el 2006

Todavía no funciona la página pero aquí estará el código para 2006

In [ ]:
read_2006 = pd.read_csv('2006_links.csv',encoding='utf-8', sep =",")

### Para el 2010

In [ ]:
read_2010 = pd.read_csv('2010_links.csv',encoding='utf-8', sep =",")

In [ ]:
f = open("politicos_db_2010.csv", "a", encoding = 'utf-8', newline = '')
writer = csv.DictWriter(
    f, fieldnames = columns)
writer.writeheader()
f.close()

In [ ]:
for politico in range(0,len(read_2010)):
    
    url = read_2010['links'][politico]
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source)

    # I. Datos Personales del Candidato

    keys_datos_personales = ['organizacion_politica', 'cargo', 'lugar_postulacion','DNI','apellido_paterno',
                            'apellido_materno','nombres','fecha_nacimiento','sexo','pais','departamento_nac',
                            'provincia_nac','distrito_nac','domicilio','departamento_dom',
                             'provincia_dom','distrito_dom', 'tiempo_dom']

    tags_datos_personales = ['lblOPolitica', 'lblCargo', 'lblUbicacion',
                        'lblDNI','lblPaterno','lblMaterno','lblNombre',
                       'lblFechaN','lblSexo','lblPais','lblRegionN',
                        'lblProvinciaN','lblDistritoN','lblResidencia',
                        'lblRegionR','lblProvinciaR', 'lblDistritoR',
                       'lblTiempoR']

    fields_datos_personales = []
    for items in range(0, len(tags_datos_personales)):
        for element in driver.find_elements_by_id(tags_datos_personales[items]):
            fields_datos_personales.append(element.text.lower())
            break
    # Las demás secciones

    datos_personales = dict(zip(keys_datos_personales, fields_datos_personales))

    partial = []
    tables = [t for t in soup.find_all('table',{"class":"tabla_hoja"}) if not t.find_all('table',{"class":"tabla_hoja"})]
    for table in range(1, len(tables)):
        my_table = tables[table]
        datos = []
        rows = my_table.findChildren(['th', 'tr'])
        for row in rows:
            try:
                cells = row.findChildren('td')
                for cell in cells:
                    try:
                        value = cell.string.strip()
                        datos.append(value)
                    except:
                        continue
            except:
                cells = my_table.findChildren(['td'])
                for cell in cells:
                    value = cell.string.strip()
                    datos.append(value)

        # Para los headers de las tablas

        rows = my_table.findChildren(['th'])
        headers = []
        for i in range (0, len(rows)):
            header = rows[i].text.replace("\n", "").strip()
            headers.append(header)

        # Haciendo una mega lista que contiene los headers seguidos de sus datos

        if len(headers) == len(datos):
            for i in range(0, len(headers)):
                partial.append(headers[i])
                partial.append(datos[i])

        elif len(headers) == 5 and len(datos) == 7:
            datos = datos[:len(datos)-2]
            for i in range(0, len(headers)):
                partial.append(headers[i])
                partial.append(datos[i])          

    # II. Experiencia laboral
    sector_publico = 'no'
    sector_privado = 'no'
    publico_periodo = 0
    privado_periodo = 0
    sector = [i for i, x in enumerate(partial) if x == "Sector"]
    fecha_desde = [i for i, x in enumerate(partial) if x == "Fecha desde"]
    fecha_hasta = [i for i, x in enumerate(partial) if x == "Fecha hasta"]


    for data in range(0,len(sector)):
        for i in range(0, len(sector)):
            if partial[sector[i] + 1] == 'Privado':
                sector_privado ='si'
            else:
                sector_publico = 'si'

    years_desde = []
    for f in range(0,len(fecha_desde)):
        e =partial[fecha_desde[f] + 1]
        years_desde.append(int(e[-4:]))


    years_hasta = []
    for f in range(0,len(fecha_hasta)):
        e =partial[fecha_hasta[f] + 1]
        try:
            years_hasta.append(int(e[-4:]))
        except:
            e = 2014
            years_hasta.append(e)

    l_periodos = []
    for year in range(0, len(years_hasta)):
        if years_hasta == 'Hasta la actualidad':
            years_hasta = 2014
        else:
            periodo_n = years_hasta[year] - years_desde[year]
            l_periodos.append(periodo_n)   

    temp_l = []
    publico_periodo = 0
    privado_periodo = 0
    for i in range(0,len(sector)):
        e =partial[sector[i] + 1].lower()
        temp_l.append(e)

    for i in range(0,len(temp_l)):
        if temp_l[i] == 'privado':
            privado_periodo = privado_periodo + l_periodos[i]
        else:
            publico_periodo = publico_periodo + l_periodos[i]

    keys_experiencia_laboral = ['trabajo_publico', 'trabajo_privado', 'publico_periodo', 'privado_periodo']
    values_experiencia_laboral = [ sector_publico, sector_privado, publico_periodo, privado_periodo]
    experiencia_laboral = dict(zip(keys_experiencia_laboral, values_experiencia_laboral))

    # III. Formación Académica

    periodo_superior = 0
    primaria = [i for i, x in enumerate(partial) if x == "Primaria"]
    secundaria = [i for i, x in enumerate(partial) if x == "Secundaria"]
    superior = [i for i, x in enumerate(partial) if x == "Documento que lo acredita"]
    periodo = [i for i, x in enumerate(partial) if x == "Período"]
    primarias = []
    secundarias = []
    superiores = []
    periodos = []

    for i in range(0, len(primaria)):
        p = partial[primaria[i]-3]
        primarias.append(p)

    for i in range(0, len(secundaria)):
        s = partial[secundaria[i]-1]
        secundarias.append(p)

    for i in range(0, len(superior)):
        sup = partial[superior[i]-1]
        superiores.append(sup)

    for i in range(0, len(periodo)):
        sup_per = partial[periodo[i]+1]
        periodos.append(sup_per)

    guion = sup_per.index("-")
    desde = int(sup_per[guion-5:guion-1])
    if sup_per[-19:] == 'Hasta la actualidad':
        hasta = 2010
    else:
        hasta = int(sup_per[-4:])

    if len(primarias) == 0:
        basica_p = 'no'
    else:
        basica_p = primarias[-1].lower()
    if len(secundarias) == 0:
        basica_s = 'no'
    else:
        basica_s = secundarias[-1].lower()
    if len(superiores) == 0:
        ed_sup = 'no'
        periodo_superior == 0
    else:
        ed_sup = superiores[-1].lower()
        periodo_superior = hasta - desde

    keys_formacion_academica = ['primaria', 'secundaria', 'superior', 'periodo_superior']    
    values_formacion_academica = [basica_p, basica_s, ed_sup, periodo_superior]
    formacion_academica = dict(zip(keys_formacion_academica,values_formacion_academica))

    # IV. Cargos Políticos

    # cargos partidarios


    flag_partidario = [i for i, x in enumerate(partial) if x == "Ámbito o circunscripción"]
    periodo_aproximado = [i for i, x in enumerate(partial) if x == "Periodo aproximado"]
    og_pol = [i for i, x in enumerate(partial) if x == "Organización política"]



    if len(flag_partidario) == 0:
        cargos_partidarios = 'no'
        op_partidarios = 'no'
        periodo_partidarios = 0

    else:

        cargos_partidario = []
        for i in range(0, len(flag_partidario)):
            cpar = partial[flag_partidario[i]-1]
            cargos_partidario.append(cpar)

        op_partidario = []
        for i in range(0, len(og_pol)):
            oppar = partial[og_pol[i]+1]
            op_partidario.append(oppar)

        periodo_partidario = []
        for i in range(0, len(periodo_aproximado)):
            ppar = partial[periodo_aproximado[i]+1]
            periodo_partidario.append(ppar)

        sep = periodo_partidario[0].lower().index(" a ")
        desde = int(periodo_partidario[0][sep-5:sep])
        hasta = int(periodo_partidario[0][-4:])

        cargos_partidarios = cargos_partidario[-1].lower()
        op_partidarios = op_partidario[-1].lower()
        periodo_partidarios = hasta - desde


    # cargos de elección popular

    flag_popular = [i for i, x in enumerate(partial) if x == "Organización por la que postulo"]

    if len(flag_popular) == 0:
        cargos_popular = 'no'
        op_popular = 'no'
        periodo_popular = 0
    else:
        cargos_popular = []
        for i in range(0, len(flag_popular)):
            cp = partial[flag_popular[i]-5]
            cargos_popular.append(cp)

        op_popular = []
        for i in range(0, len(flag_popular)):
            opp = partial[flag_popular[i]+1]
            op_popular.append(opp)

        periodo_popular = []
        for i in range(0, len(flag_popular)):
            pp = partial[flag_popular[i]+3]
            periodo_popular.append(pp)

        desde = int(periodo_popular[0][:4])
        hasta = int(periodo_popular[0][-4:])


        cargos_popular = cargos_popular[-1].lower()
        op_popular = op_popular[-1].lower()
        periodo_popular = hasta - desde


    keys_cargos_politicos = ['organizacion_politica_partidarios','cargo_partidarios','periodo_partidarios','organizacion_politica_eleccion_popular', 'cargo_eleccion_popular','periodo_eleccion_popular']
    values_cargos_politicos = [op_partidarios, cargos_partidarios,periodo_partidarios,op_popular,cargos_popular,periodo_popular]
    cargos_politicos = dict(zip(keys_cargos_politicos,values_cargos_politicos))

    row = {**datos_personales, **experiencia_laboral, **formacion_academica,**cargos_politicos}

    driver.quit()

    print(politico)

    with open("politicos_db_2010.csv", "a", encoding="utf-8", newline = '') as outfile:
        csvwriter = csv.writer(outfile)
        csvwriter.writerow(row.values())

#### Exportando a Excel

In [ ]:
read_file = pd.read_csv ('politicos_db_2010.csv')
read_file.to_excel ('politicos_db_2010.xlsx', index = None, header=True)

### Para el 2014

In [ ]:
read_2014 = pd.read_csv('2014_links.csv',encoding='utf-8', sep =",")

#### Añadiendo cabeceras a las base de datos

In [ ]:
f = open("politicos_db_2014.csv", "a",encoding="utf-8", newline = '')
writer = csv.DictWriter(
    f, fieldnames = columns)
writer.writeheader()
f.close()

In [ ]:
for politico in range(0,len(read_2014)):
    
    url = read_2014['links'][politico]   
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)

    # I. Datos Personales del Candidato


    keys_datos_personales = ['organizacion_politica', 'cargo', 'lugar_postulacion','DNI','apellido_paterno',
                            'apellido_materno','nombres','fecha_nacimiento','sexo','pais','departamento_nac',
                            'provincia_nac','distrito_nac','domicilio','departamento_dom',
                             'provincia_dom','distrito_dom', 'tiempo_dom']

    tags_datos_personales = ['txtOrganizacionPolitica', 'txtCargoPostula', 'txtLugarPostula',
                        'txtDNI','txtApellidoPaterno','txtApellidoMaterno','txtNombres',
                       'txtFechaNacimiento','txtSexo','txtPais','txtDepartamentoNac',
                        'txtProvinciaNac','txtDistritoNac','txtLugarResicencia',
                        'txtLugarDepartamentoRes','txtLugarProvinciaRes', 'txtLugarDistritoRes',
                       'txtTiempoRes']

    fields_datos_personales = []
    for items in range(0, len(tags_datos_personales)):
        for element in driver.find_elements_by_id(tags_datos_personales[items]):
            fields_datos_personales.append(element.text)
            break

    datos_personales = dict(zip(keys_datos_personales, fields_datos_personales))


    # II. Experiencia laboral

    partial = []
    table = [t for t in soup.find_all('table', {'id':"tblExperiencia"}) if not t.find_all('table',{'id':"tblExperiencia"})]
    if len(table) == 0:
        sector_publico = 'no'
        sector_privado = 'no'
        publico_periodo = 0
        privado_periodo = 0
        
        keys_experiencia_laboral = ['trabajo_publico', 'trabajo_privado', 'publico_periodo', 'privado_periodo']
        values_experiencia_laboral = [ sector_publico, sector_privado, publico_periodo, privado_periodo]
        experiencia_laboral = dict(zip(keys_experiencia_laboral, values_experiencia_laboral)) 
    else:
        
        my_table = table[0]
        datos = []
        rows = my_table.findChildren(['th','tr'])
        cells = my_table.findChildren('td')
        for cell in cells:
            try:
                value = cell.string.strip()
                datos.append(value)
            except:
                continue

        rows = my_table.findChildren(['th'])
        headers = []
        for i in range (0, len(rows)):
            header = rows[i].text.replace("\n", "").strip()
            headers.append(header)


        if len(datos) - len(headers) != 0:
            datos1 = [x for x in datos if x != '']
            for i in range(0, len(headers)):
                partial.append(headers[i])
                partial.append(datos1[i])
           
        else:
            for i in range(0, len(headers)):
                    partial.append(headers[i])
                    partial.append(datos[i])


        sector_publico = 'no'
        sector_privado = 'no'
        publico_periodo = 0
        privado_periodo = 0
        sector = [i for i, x in enumerate(partial) if x == "Sector"]
        fecha_desde = [i for i, x in enumerate(partial) if x == "Fecha desde"]
        fecha_hasta = [i for i, x in enumerate(partial) if x == "Fecha hasta"]


        for data in range(0,len(sector)):
            for i in range(0, len(sector)):
                if partial[sector[i] + 1] == 'Privado':
                    sector_privado ='si'
                else:
                    sector_publico = 'si'

            years_desde = []
            for f in range(0,len(fecha_desde)):
                e =partial[fecha_desde[f] + 1]
                years_desde.append(e)


            years_hasta = []
            for f in range(0,len(fecha_hasta)):
                e =partial[fecha_hasta[f] + 1]
                years_hasta.append(e)

            l_periodos = []
            for year in range(0, len(years_hasta)):
                if years_hasta[year] == 'Hasta la actualidad':
                    years_hasta[year] = 2014
                    periodo_n = years_hasta[year] - int(years_desde[year])
                    l_periodos.append(periodo_n)   
                else:
                    periodo_n = int(years_hasta[year]) - int(years_desde[year])
                    l_periodos.append(periodo_n)   

            temp_l = []
            publico_periodo = 0
            privado_periodo = 0
            for i in range(0,len(sector)):
                e =partial[sector[i] + 1].lower()
                temp_l.append(e)

            for i in range(0,len(temp_l)):
                if temp_l[i] == 'privado':
                    privado_periodo = privado_periodo + l_periodos[i]
                else:
                    publico_periodo = publico_periodo + l_periodos[i]


        keys_experiencia_laboral = ['trabajo_publico', 'trabajo_privado', 'publico_periodo', 'privado_periodo']
        values_experiencia_laboral = [ sector_publico, sector_privado, publico_periodo, privado_periodo]
        experiencia_laboral = dict(zip(keys_experiencia_laboral, values_experiencia_laboral))  

    # III. Formación Acadmémica

    #Educación Primaria

    partial = []
    table = [t for t in soup.find_all('table', {'id':"tblEducacionPrimaria"}) if not t.find_all('table',{'id':"tblEducacionPrimaria"})]
    my_table = table[0]
    datos = []
    rows = my_table.findChildren(['th','tr'])
    cells = my_table.findChildren('td')
    for cell in cells:
        try:
            value = cell.string.strip()
            datos.append(value)
        except:
            continue

    rows = my_table.findChildren(['th'])
    headers = []
    for i in range (0, len(rows)):
        header = rows[i].text.replace("\n", "").strip()
        headers.append(header)


    if len(datos) - len(headers) != 0:
        datos1 = [x for x in datos if x != '']
        for i in range(0, len(headers)):
            partial.append(headers[i])
            partial.append(datos1[i])
           
    else:
        for i in range(0, len(headers)):
                partial.append(headers[i])
                partial.append(datos[i])

    primaria = [i for i, x in enumerate(partial) if x == "Concluido"]

    if len(primaria) == 0:
        basica_p = 'no'
    else:
        basica_p = 'si'

    # Educación Secundaria

    partial = []
    table = [t for t in soup.find_all('table', {'id':"tblEducacionSecundaria"}) if not t.find_all('table',{'id':"tblEducacionSecundaria"})]
    my_table = table[0]
    datos = []
    rows = my_table.findChildren(['th','tr'])
    cells = my_table.findChildren('td')
    for cell in cells:
        try:
            value = cell.string.strip()
            datos.append(value)
        except:
            continue

    rows = my_table.findChildren(['th'])
    headers = []
    for i in range (0, len(rows)):
        header = rows[i].text.replace("\n", "").strip()
        headers.append(header)


    if len(datos) - len(headers) != 0:
        datos1 = [x for x in datos if x != '']
        for i in range(0, len(headers)):
            partial.append(headers[i])
            partial.append(datos1[i])

    else:
        for i in range(0, len(headers)):
                partial.append(headers[i])
                partial.append(datos[i])

    secundaria = [i for i, x in enumerate(partial) if x == "Concluido"]

    if len(secundaria) == 0:
        basica_s = 'no'
    else:
        basica_s = 'si'

    # Educación Superior 

    partial1 = []
    table = [t for t in soup.find_all('table', {'id':"tblTecnico"}) if not t.find_all('table',{'id':"tblTecnico"})]
    my_table = table[-1]
    datos = []
    rows = my_table.findChildren(['th','tr'])
    cells = my_table.findChildren('td')
    for cell in cells:
        try:
            value = cell.string.strip()
            datos.append(value)
        except:
            continue
            
    rows = my_table.findChildren(['th'])
    headers = []
    for i in range (0, len(rows)):
        header = rows[i].text.replace("\n", "").strip()
        headers.append(header)


    if len(datos) - len(headers) != 0:
        datos1 = [x for x in datos if x != '']
        for i in range(0, len(headers)):
            partial1.append(headers[i])
            partial1.append(datos1[i])

    else:
        for i in range(0, len(headers)):
                partial1.append(headers[i])
                partial1.append(datos[i])

    tecnico = [i for i, x in enumerate(partial1) if x == "Estado"]
    periodo_tecnico = [i for i, x in enumerate(partial1) if x == "Periodo"]

    partial2 = []
    table = [t for t in soup.find_all('table', {'id':"tblUniversitario"}) if not t.find_all('table',{'id':"tblUniversiario"})]
    my_table = table[-1]
    datos = []
    rows = my_table.findChildren(['th','tr'])
    cells = my_table.findChildren('td')
    for cell in cells:
        try:
            value = cell.string.strip()
            datos.append(value)
        except:
            continue

    rows = my_table.findChildren(['th'])
    headers = []
    for i in range (0, len(rows)):
        header = rows[i].text.replace("\n", "").strip()
        headers.append(header)


    if len(datos) - len(headers) != 0:
        datos1 = [x for x in datos if x != '']
        for i in range(0, len(headers)):
            partial2.append(headers[i])
            partial2.append(datos1[i])
           
    else:
        for i in range(0, len(headers)):
                partial2.append(headers[i])
                partial2.append(datos[i])
                
    universitario = [i for i, x in enumerate(partial2) if x == "Estado"]
    periodo_universitario = [i for i, x in enumerate(partial2) if x == "Periodo" or x == "Período"]
    
    años = []
    if len(universitario) == 0 and len(tecnico) == 0:
        ed_sup = 'no'
        periodo_superior = 0
    if len(universitario) > 0:
        ed_sup = 'si'
        try:
            ans  = partial2[periodo_universitario[-1] +1]
            años.append(ans)
            guion = años[-1].index("-")
            hasta = años[-1][guion+2:11]
            if hasta == "Hast":
                hasta = 2014
                desde = int(años[-1][:4])
                periodo_superior = hasta - desde
            else:
                desde = int(años[-1][:4])
                periodo_superior = int(hasta) - desde
        except:
            ans  = partial2[periodo_universitario[-1] -1]
            años.append(ans)
            guion = años[-1].index("-")
            hasta = años[-1][guion+2:11]
            if hasta == "Hast":
                hasta = 2014
                desde = int(años[-1][:4])
                periodo_superior = hasta - desde
            else:
                desde = int(años[-1][:4])
                periodo_superior = int(hasta) - desde
    if len(tecnico) > 0:
        ed_sup = 'si'
        ans  = partial1[periodo_tecnico[-1] +1]
        años.append(ans)
        guion = años[-1].index("-")
        hasta = int(años[-1][guion+2:11])
        desde = int(años[-1][:4])
        periodo_superior = hasta - desde

    keys_formacion_academica = ['primaria', 'secundaria', 'superior', 'periodo_superior']    
    values_formacion_academica = [basica_p, basica_s, ed_sup, periodo_superior]
    formacion_academica = dict(zip(keys_formacion_academica,values_formacion_academica))


    # IV. Cargos Políticos

    # cargos partidarios

    partial = []
    table = [t for t in soup.find_all('table', {'id':"tblCargoPartidario"}) if not t.find_all('table',{'id':"tblCargoPartidario"})]
    my_table = table[0]
    datos = []
    rows = my_table.findChildren(['th','tr'])
    cells = my_table.findChildren('td')
    for cell in cells:
        try:
            value = cell.string.strip()
            datos.append(value)
        except:
            continue

    rows = my_table.findChildren(['th'])
    headers = []
    for i in range (0, len(rows)):
        header = rows[i].text.replace("\n", "").strip()
        headers.append(header)


    if len(datos) - len(headers) != 0:
        datos1 = [x for x in datos if x != '']
        for i in range(0, len(headers)):
            partial.append(headers[i])
            partial.append(datos1[i])

    else:
        for i in range(0, len(headers)):
                partial.append(headers[i])
                partial.append(datos[i])

    cargos_partida = [i for i, x in enumerate(partial) if x == "Cargo"]
    op_partida = [i for i, x in enumerate(partial) if x == "Organización Política"]
    periodo_partida = [i for i, x in enumerate(partial) if x == "Periodo"]

    años = []
    if len(cargos_partida) == 0:
        cargos_partidarios = 'no'
    else:
        cargos_partidarios = partial[cargos_partida[-1] +1].lower()

    if len(cargos_partida) == 0:
        op_partidarios = 'no'
    else:
        op_partidarios = partial[op_partida[-1] +1].lower()

    if len(periodo_partida) == 0:
        periodo_partidarios = 0
    else:
        ans  = partial[periodo_partida[-1] +1]
        años.append(ans)
        guion = años[-1].index("-")
        hasta = int(años[-1][guion+2:11])
        desde = int(años[-1][:4])
        periodo_partidarios = hasta - desde


    # cargos de elección popular

    partial = []
    table = [t for t in soup.find_all('table', {'id':"tblCargoEleccion"}) if not t.find_all('table',{'id':"tblCargoEleccion"})]
    my_table = table[0]
    datos = []
    rows = my_table.findChildren(['th','tr'])
    cells = my_table.findChildren('td')
    for cell in cells:
        try:
            value = cell.string.strip()
            datos.append(value)
        except:
            continue

    rows = my_table.findChildren(['th'])
    headers = []
    for i in range (0, len(rows)):
        header = rows[i].text.replace("\n", "").strip()
        headers.append(header)


    if len(datos) - len(headers) != 0:
        datos1 = [x for x in datos if x != '']
        for i in range(0, len(headers)):
            partial.append(headers[i])
            partial.append(datos1[i])

    else:
        for i in range(0, len(headers)):
                partial.append(headers[i])
                partial.append(datos[i])



    cargos_popula = [i for i, x in enumerate(partial) if x == "Cargo"]
    op_popula = [i for i, x in enumerate(partial) if x == "Organizacion Política por la que postuló"]
    periodo_popula = [i for i, x in enumerate(partial) if x == "Periodo"]
    años = []
    if len(cargos_popula) == 0:
        cargos_popular = 'no'
    else:
        cargos_popular = partial[cargos_popula[-1] +1]

    if len(cargos_popula) == 0:
        op_popular = 'no'
    else:
        op_popular = partial[op_popula[-1] +1]

    if len(periodo_popula) == 0:
        periodo_popular = 0
    else:
        ans  = partial[periodo_popula[-1] +1]
        años.append(ans)
        guion = años[-1].index("-")
        hasta = años[-1][guion+2:11]
        desde = int(años[-1][:4])
        if hasta == "Hast":
            hasta = 2014
            periodo_popular = hasta - desde
        else:
            periodo_popular = int(hasta) - desde
            
    keys_cargos_politicos = ['organizacion_politica_cargos_partidarios','cargo_partidarios','periodo_cargos_partidarios',
                        'organizacion_politica_eleccion_popular', 'cargo_eleccion_popular','periodo_eleccion_popular']
    values_cargos_politicos = [op_partidarios,periodo_partidarios,op_popular,cargos_popular,periodo_popular]

    cargos_politicos = dict(zip(keys_cargos_politicos,values_cargos_politicos))


    driver.quit()  
    
    row = {**datos_personales, **experiencia_laboral, **formacion_academica,**cargos_politicos}
    
    print(politico)
    
    with open("politicos_db_2014.csv", "a", encoding="utf-8", newline = '') as outfile:
        csvwriter = csv.writer(outfile)
        csvwriter.writerow(row.values())

#### Exportando a Excel

In [ ]:
read_file = pd.read_csv ('politicos_db_2014.csv')
read_file.to_excel ('politicos_db_2014.xlsx', index = None, header=True)